## Import packages & dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Input image
Here, say we assume a single grayscale image of size 8x8.
Input image size (h): 8x8x1

In [2]:
X = np.random.randn(8,8)

## Convolution layer

In [4]:
# define filters here, of dim 3x3x2
num_filters = 2
filter_size = 3

filters = np.random.randn(num_filters, filter_size, filter_size)
bias = np.zeros(num_filters)

# convolution operation
def convolve(X, filters, bias):
    h, w = X.shape
    num_filters, f, _ = filters.shape

    output_dim = h - f + 1
    out = np.zeros((num_filters, output_dim, output_dim))

    for k in range(num_filters):
        for i in range(output_dim):
            for j in range(output_dim):
                region = X[i:i+f, j:j+f]
                out[k, i, j] = np.sum(region * filters[k]) + bias[k]

    return out

conv_out = convolve(X, filters, bias)

## Relu Activation

In [6]:
def relu(X):
    return np.maximum(0, X)

relu_out = relu(conv_out)


## Max pooling

In [7]:
def max_pool(X, size=2, stride=2):
    num_filters, h, w = X.shape
    out_dim = h // stride
    out = np.zeros((num_filters, out_dim, out_dim))

    for k in range(num_filters):
        for i in range(0, h, stride):
            for j in range(0, w, stride):
                region = X[k, i:i+size, j:j+size]
                out[k, i//stride, j//stride] = np.max(region)

    return out

pool_out = max_pool(relu_out)

## Flatten

In [9]:
flattened = pool_out.reshape(-1)


## Fully Connected Layer

In [10]:
# initialize weights
num_classes = 2

W_fc = np.random.randn(18, num_classes)
b_fc = np.zeros(num_classes)

# Forward pass
logits = flattened @ W_fc + b_fc

## Softmax probabilities

In [11]:
def softmax(z):
    exp = np.exp(z - np.max(z))
    return exp / np.sum(exp)

probs = softmax(logits)

## Prediction

In [12]:
prediction = np.argmax(probs)

## Loss function (Cross-Entropy)

In [13]:
y_true = 1
loss = -np.log(probs[y_true])
